# Mapas de locaciones con eventos diferente a viaje

Estos eventos son todos los que 'tipoevento' es diferente a 4 (Viaje Normal). 
Los eventos a mapearse son:

8. Devolución 
10. Recarga de la tarjeta. 
14. Devolucion

Importar librerías que vamos a usar

In [1]:
import pandas as pd
import numpy as np


import geopandas as gpd
import contextily as cx

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

Ahora vamos a cargar los archivos de datos que vamos a utlizar

In [2]:
datos_eventos = pd.read_csv("../../data/processed/consolidado-full-sin-viajes.csv", sep = ';',low_memory=False)
print(datos_eventos.sample(10))

                           serialtarjeta           idsam   
116926  61942dce1c316065fc014bfbe72e4f58  04351722b95a80  \
196128  186ed19641a06c6dbafdba18968d1601  04301a22b95a80   
169966  2349bd355c30b670ecb1b71e1060af27  04191fe24e6180   
148714  324de917ab615d81280abd61b8d9af4d  04352622b95a80   
83944   a333c7c2eb25e50ee57a5e35724c3ad1  04391842846280   
285210  fca2345e11c6852b29d716b711e8ca2a  04202fd2955d80   
341544  2f6bfa8bfde9682b7d81bb16a14c9c6d  041f3dd2955d80   
67432   b3fecf25e7e89d48df77997308fd2d93  0412281aaa6380   
334979  838c160606537f554f0dc89ff42e1532  0412321aaa6380   
41656   bab159c3f0fe583811a6ee298945c89e  042035d2955d80   

                      fechahoraevento producto  montoevento   
116926  2022/12/13 12:18:26.000000000       MO         3400  \
196128  2023/03/17 14:01:30.000000000       MO         3400   
169966  2023/02/14 05:25:48.000000000       MO         2300   
148714  2023/01/11 09:51:32.000000000       MO         3400   
83944   2022/11/17 19:49

Con los datos cargados, comenzamos el trabajo de limpieza y preparación para poder procesarlo y generar un mapa de eventos.

In [3]:
datos_eventos.dropna(how='any', inplace=True) # BORRAR NaNs

datos_eventos['latitude']=datos_eventos['latitude'].astype('float')
datos_eventos['longitude']=datos_eventos['longitude'].astype('float')

datos_eventos = datos_eventos[(datos_eventos[['latitude','longitude']] != 0).all(axis=1)] # Remover ceros
datos_eventos = datos_eventos[datos_eventos['latitude'] < -25.05]  # Remover latitudes fuera de Asunción
datos_eventos = datos_eventos[datos_eventos['longitude'] < -57.2] # Remover longitudes fuera de Asunción
datos_eventos = datos_eventos[datos_eventos['latitude'] > -25.8]  # Remover latitudes fuera de Asunción
# datos_eventos = datos_eventos[datos_eventos['latitude'] > -58]  # Remover latitudes fuera de Asunción



datos_eventos["tipotransporte_cat"] = pd.cut(
    x=datos_eventos["tipotransporte"],
    bins=[0, 1,3, np.inf],
    labels=["Bus municipal interno", "Normal", "Diferencial"],
)

# Se refiere al propietario del validador donde: - 1. VMT - 2. MAS - 3. JAHA
datos_eventos["identidad_cat"] = pd.cut(
    x=datos_eventos["identidad"],
    bins=[1, 2, 3, np.inf],
    labels=["VMT", "MAS", "JAHA"],
)


print(datos_eventos.sample(10))

                           serialtarjeta           idsam   
61077   f064870210c318c5db4987ae209482f0  041e31e24e6180  \
338591  2a653f71dbf44616210afcff8a513cd8  04352fd2955d80   
153837  f2d27e5faa3876a3ac5d5e5d8d8e9564  0413231aaa6380   
49487   6532528d96280db5a16c3d9eec50f417  04143922626c80   
324248  be57724acf7bb894c408068c08166efc  041b18e24e6180   
130955  1ea75acdd93babd7be318739009a5592  041d0ee24e6180   
292379  98ce28e69c2f16220e2da23e5cf96e1d  043e2142846280   
40901   3161cc048a1547ded0f08d6b97773a8c  043610d2955d80   
276064  21563cd117eb7a36bbba64fa5d06730a  041820e24e6180   
36095   c970efe57ce7f02c98222727bf253b5b  043618d2955d80   

                      fechahoraevento producto  montoevento   
61077   2022/10/13 15:15:28.000000000       MO         2300  \
338591  2022/07/04 14:40:46.000000000       MO         2300   
153837  2023/01/05 07:11:42.000000000       MO         2800   
49487   2022/10/25 07:41:52.000000000       MO         2800   
324248  2022/07/21 15:05

Ahora con los datos listos comenzamos a trabajar generando los mapas.
Vamos a generar diferentes mapas con estos datos: 
- serialtarjeta
- idsam
- producto
- identidad
- idrutaestacion
- tipotransporte

In [4]:
print("=====> Generando gráficos")

latlon_por_evento= gpd.GeoDataFrame(datos_eventos, geometry=gpd.points_from_xy(datos_eventos.longitude,datos_eventos.latitude),crs="EPSG:4326")


print("Generando gráfico por producto")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='producto',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para producto')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/producto.png',dpi=300)
plt.close()
print("Gráfico por producto listo")


print("Generando gráfico por identidad")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='identidad_cat',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para identidad')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/identidad.png',dpi=300)
plt.close()
print("Gráfico por identidad listo")


print("Generando gráfico por idrutaestacion")
eventos_mapa_plot = latlon_por_evento.plot(legend=False,column='idrutaestacion',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para idrutaestacion')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/idrutaestacion.png',dpi=300)
plt.close()
print("Gráfico por idrutaestacion listo")


# Tipo de transporte donde - 0. Bus municipal interno - 1. Normal - 3. Diferencial
print("Generando gráfico por tipotransporte")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='tipotransporte_cat',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para tipotransporte')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/tipotransporte.png',dpi=300)
plt.close()
print("Gráfico por tipotransporte listo")


print("Generando gráfico por idsam")
eventos_mapa_plot = latlon_por_evento.plot(legend=False,column='idsam',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos por idsam')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/idsam.png',dpi=300)
plt.close()
print("Gráfico por idsam listo")


print("Generando gráfico por serialtarjeta")
eventos_mapa_plot = latlon_por_evento.plot(legend=False,column='serialtarjeta',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos por serialtarjeta')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/serialtarjeta.png',dpi=300)
plt.close()
print("Gráfico por serialtarjeta listo")

print("Gráficos generados <=====")

=====> Generando gráficos
Generando gráfico por producto
Gráfico por producto listo
Generando gráfico por identidad
Gráfico por identidad listo
Generando gráfico por idrutaestacion
Gráfico por idrutaestacion listo
Generando gráfico por tipotransporte
Gráfico por tipotransporte listo
Generando gráfico por idsam
Gráfico por idsam listo
Generando gráfico por serialtarjeta
Gráfico por serialtarjeta listo
Gráficos generados <=====
